##**Activar Google Drive en el entorno**

Se recomienda el uso de Google Drive para el almacenamiento de datos y el entrenamiento de StyleGAN2. Al ejecutar esta celda, sigue los pasos indicados en la consola.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##**Librerías**##

En la siguiente celda se importan todas las librerías necesarias.

In [ ]:
import tensorflow as tf
import torch
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import os, sys
import cv2 as cv
from scipy import ndimage, misc
import imageio
from PIL import Image
import PIL

**GPU**

Este comando obtiene información acerca de la GPU usada para el entrenamiento y evaluación de StyleGAN2

In [ ]:
!nvidia-smi

**Modelo**

Descarga del modelo desde el GitHub de Nvidia.

In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
!pip install ninja

##**Datos de entrenamiento**##

Para entrenar a StyleGAN2 se necesita un dataset con imágenes que sean cuadradas y en modo de color RGB. Sin embargo, se puede usar cualquier dataset, ya que en esta sección se podrán corregir o eliminar las imágenes que no cumplan los requisitos.

In [ ]:
!git clone https://github.com/cardstdani/WasteClassificationNeuralNetwork.git

**Redimensionar las imágenes de nuestro conjunto de datos**

Esta celda copia todas las imágenes del dataset a una carpeta que será usada durante el resto del entrenamiento. Además, redimensiona las imágenes a un tamaño estándar para todo el dataset, en este caso es (256, 256). <br><br>
**Recuerda:** El tamaño de las imágenes puede incrementar significativamente el tiempo de entrenamiento, ya que es un modelo complejo.

In [ ]:
!mkdir dataset #Crear la carpeta donde se guardará el conjunto de datos

outPath = "/content/dataset" #Ruta de salida
path = "/content/WasteClassificationNeuralNetwork/WasteImagesDataset" #Ruta donde se encuentra nuestro dataset (ES NECESARIO COMPROBAR ESTA RUTA)
dim = (256, 256) #Dimensiones de salida para todas las imágenes del dataset

for p in os.scandir(path):
  if p.is_dir():
    for image_path in os.listdir(p):
      image_to_modify = imageio.imread(os.path.join(p, image_path))
      changed = np.array(Image.fromarray(image_to_modify).resize(dim, Image.BILINEAR)).astype(np.double)
      fullpath = os.path.join(outPath, 'out_'+image_path)
      imageio.imwrite(fullpath, changed)
  else:
    image_to_modify = imageio.imread(p.path)
    changed = np.array(Image.fromarray(image_to_modify).resize(dim, Image.BILINEAR)).astype(np.double)
    fullpath = os.path.join(outPath, 'out_'+p.name)
    imageio.imwrite(fullpath, changed)

**Comprobación de las imágenes**

Este paso es uno de los más importantes. Se comprueba todas las imágenes del dataset, y todas las que no sean cuadradas o estén en modo de color RGB son eliminadas.

In [ ]:
from os import listdir
from os.path import join

imgNumber = 0
base_size = None
for file in os.listdir(outPath):
  imgNumber += 1
  file2 = os.path.join(outPath,file)
  img = Image.open(file2)
  sz = img.size
  
  #Eliminar todas las imágenes erróneas
  if base_size and sz!=base_size:
    print(f"Tamaño inconsistente: {file2}")
    os.remove(file2)
    imgNumber -= 1
  elif img.mode!='RGB':
    print(f"El formato de color no es RGB: {file2}")
    os.remove(file2)
    imgNumber -= 1
  else:
    base_size = sz

print("El dataset contiene: " + str(imgNumber) + " imágenes")

##**Entrenamiento**##

En esta sección se entrena el modelo.

In [ ]:
outputDir = "/content/drive/MyDrive/StyleGAN2/WasteClassification/" #carpeta en la que se guardan los resultados del entrenamiento (se recomienda guardar en drive)

snapshot_count = 1 #Variable de autoguardado, es decir, cada n iteraciones se guarda el modelo y los resultados en la carpeta de salida
mirrored = True #Reflejar las imágenes horizontalmente (True/False)
metric_list = None #Lista de métricas, por defecto en None
numGPUS = 1 #Número de tarjetas gráficas usadas para el entrenamiento, aquí en Google Colab por defecto está en 1
augs = "bg" #Canal de aumento de las imágenes, por defecto en "bg"

**Para empezar desde 0 el entrenamiento**

Esta celda debe usarse solo cuando se comienza a entrenar un modelo desde 0, si ya tienes un modelo preentrenado y quieres continuar, ve a la celda siguiente.

In [ ]:
!python /content/stylegan2-ada-pytorch/train.py --outdir {outputDir} --gpus={numGPUS} --snap={snapshot_count} --data={outPath} --augpipe={augs} --mirror={mirrored} --metrics={metric_list}

**Para continuar con un entrenamiento previo**

Los modelos generados por StyleGAN2 se guardan en formato .pkl

In [ ]:
resume_from = "/content/drive/MyDrive/StyleGAN2/WasteClassification/00006-dataset-mirror-auto1-bg/network-snapshot-000004.pkl" #Ruta del modelo con el que se reanudará el entrenamiento (ES NECESARIO COMPROBAR ESTA RUTA)

!python /content/stylegan2-ada-pytorch/train.py --outdir {outputDir} --gpus={numGPUS} --snap={snapshot_count} --data={outPath} --augpipe={augs} --mirror={mirrored} --metrics={metric_list} --resume={resume_from}

##**Resultados**##

**Cargar modelo y funciones**

In [ ]:
sys.path.insert(0, "/content/stylegan2-ada-pytorch")
import pickle
from IPython.display import Image
import IPython.display
import dnnlib
!pip install legacy
import legacy

def seed2vec(G, seed):
  return np.random.RandomState(seed).randn(1, G.z_dim)

def get_label(G, device, class_idx):
  label = torch.zeros([1, G.c_dim], device=device)
  if G.c_dim != 0:
      if class_idx is None:
          ctx.fail('Must specify class label with --class when using a conditional network')
      label[:, class_idx] = 1
  else:
      if class_idx is not None:
          print ('warn: --class=lbl ignored when running on an unconditional network')
  return label

def generate_image(device, G, z, truncation_psi=1.0, noise_mode='const', class_idx=None):
  z = torch.from_numpy(z).to(device)
  label = get_label(G, device, class_idx)
  img = G(z, label, truncation_psi=truncation_psi, noise_mode=noise_mode)
  img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
  return PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB')

device = torch.device('cuda')
modelPath = "/content/drive/MyDrive/StyleGAN2/WasteClassification/00004-dataset-mirror-auto1-bg-resumecustom/network-snapshot-000176.pkl" #Ruta del modelo que vamos a probar (ES NECESARIO COMPROBAR ESTA RUTA)
with open(modelPath, 'rb') as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device)

**Generar imágenes en un rango de "semillas"**

Las semillas son números que hacen la función de "identificadores" para todas las imágenes que puede generar el modelo. Esta celda se encarga de generar y mostrar todas las imágenes que existen entre la "semilla" de inicio y final.

In [ ]:
SEED_FROM = 1000 #Semilla de inicio
SEED_TO = 1002 #Semilla final

# Se muestran todas las imágenes que el modelo genera entre los valores de la semilla de inicio y final
for i in range(SEED_FROM, SEED_TO):
  print("Seed: " + str(i))
  z = seed2vec(G, i)
  img = generate_image(device, G, z)
  plt.axis('off')
  plt.imshow(img)
  plt.show()

**Generar un vídeo con una interpolación de imágenes**

En este caso, en vez de simplemente generar y mostrar las imágenes, se crea un vídeo con la interpolación de las imágenes generadas y se guarda en el entorno.

In [ ]:
def expand_seed(seeds, vector_size):
  result = []

  for seed in seeds:
    rnd = np.random.RandomState(seed)
    result.append( rnd.randn(1, vector_size) ) 
  return result

vector_size = G.z_dim

SEEDS = [10, 15, 12] #Configura el número de semillas (comienzo, semilla intermedia, final)
STEPS = 100 #Número de pasos por cada semilla

# Borrar resultados previos (si hay alguno)
!rm /content/resultados/* 

from tqdm.notebook import tqdm

os.makedirs("./resultados/", exist_ok=True) #Crear una carpeta llamada resultados donde se almacenarán los frames del vídeo

# Generar y guardar en la carpeta "resultados" todos los frames del vídeo
idx = 0
for i in range(len(SEEDS)-1):
  v1 = seed2vec(G, SEEDS[i])
  v2 = seed2vec(G, SEEDS[i+1])

  step = (v2 - v1) / STEPS
  current = v1.copy()

  for j in tqdm(range(STEPS), desc=f"Seed {SEEDS[i]}"):
    current = current + step
    img = generate_image(device, G, current)
    img.save(f'./resultados/frame-{idx}.png')
    idx+=1
 
outputVideoName = "output_video" #Nombre del vídeo de salida
!ffmpeg -r 30 -i /content/resultados/frame-%d.png -vcodec mpeg4 -y {outputVideoName}.mp4